In [ ]:
import pandas as pd
import numpy as np
from lmfit import minimize, Parameters
import scipy.integrate as scint
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [ ]:
def bee_eq(y, t, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, 
           eta_A_W, eta_B_W, theta_ideal):
    hive_temp = y
    M = -h*(w1*w2*(1-np.exp(-hive_temp+theta_ideal)) / (w2+w1*np.exp(-hive_temp+theta_ideal)))
    W = 4*(k_W+k_hive)*(-hive_temp + env_temp) + 4*eta_W*solar 
    A_W = (k_A_W+k_hive)*(-hive_temp + env_temp) + eta_A_W*solar
    B_W = (k_W+k_hive)*(-hive_temp + env_temp) + eta_B_W*solar
    dydt = M + W + A_W + B_W
    return dydt

def run_bee_eq(t, a, w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal):
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, env_temp,h, solar, k_W, k_A_W, k_hive, 
                                           eta_W, eta_A_W, eta_B_W, theta_ideal), 
                                           col_deriv = True, rtol = 10e-3, atol = 10e-3)
                                           #hmin = 0.001) #w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t    

In [ ]:
def residual(ps, ts, data, l, argv, k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W):
    d = pd.DataFrame(data).groupby(data['Date'])
    model = []
    k = 0
    alpha  = 1
    r_c = 115 #29#367 #ps['w1'].value
    r_h = 1094 #280#1151 #ps['w2'].value
    h_ = [1]
    return_value = []
    for m,n in d:
        h = ps['h_'+str(k)].value #health factor per day
        theta_ideal = ps['theta_'+str(k)].value + 273.15 #theta_ideal per day
        t_max = len(n[argv[1]])-1
        t = np.linspace(0,t_max, num = t_max+1)
        environment_temp = n[argv[1]].values + 273.15 #hourly environment temperature -> kelvin
        solar_rad = n[argv[2]].values #hourly solar radiation -> kelvin
        
        a = [1]*len(environment_temp)
        
        fitted = run_bee_eq(t, a, r_c, r_h, environment_temp, h, solar_rad, k_W, k_A_W, 
                            k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal)
        model = np.concatenate((model, fitted - 273.15))
        try:
            h_.append(abs(h - ps['h_'+str(k-1)].value))
        except:
            h_.append(abs(h_[k] - h))
        k = k+1
    return_value = np.concatenate((return_value, (model - data[argv[3]]).ravel()))
    return_value = np.concatenate((return_value, l*np.array(h_[2:]).ravel()))
    #print(np.mean(abs((model - data[argv[3]]).ravel())))
    return return_value

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
def main(*argv):
    data = pd.read_csv(argv[0])
    d = pd.DataFrame(data).groupby([data['Date']])
    k_W = 1 #thermal conductivity of wood -> per hour
    eta_W = 0.0*k_W #heat absorption coeff of wood
    k_A_W = k_W #thermal conductivity of top surface (wood + aluminum plate) -> per hour
    eta_A_W = eta_W #heat absorption coeff of top surface
    eta_B_W = eta_W #heat absorption coeff of bottom surface
    k_hive = 1
    
    h_max = 1
    l = 96/h_max

    forecasted_all = []

    for i in range(0,int(len(data)/24) - 21 + 1):
        print('hello', i)
        data_all = data[i*24:24*(i+21)] #input + forecast -> 8 days
        data_input =  data[i*24:24*(i+7)] #7 days input for forecasting
        data_temp_input = data_input[argv[3]][0:24*7] #7 days hive core temperature for forecasting -> celcius
        data_ext_input = data_input[argv[1]][0:24*7] #7 days environment temperature for forecasting -> celcius
        print(data_input['Date'][0:24*7])
        params = Parameters()
        for j in range(7):
            params.add('h_'+str(j), value = 0.5, min = 0.2, max = h_max)
            params.add('theta_'+str(j), value = 35, min = 33, max = 38)   
        t_max = len(data_ext_input)-1
        t = np.linspace(0,t_max, num = t_max+1)
        result = minimize(residual, params, args=(t,data_input, l, argv, k_W, k_A_W, 
                                                  k_hive, eta_W, eta_A_W, eta_B_W),
                          method='leastsq',nan_policy='omit',max_nfev = 300)
        sig_min = data_temp_input + result.residual[:len(data_temp_input)].reshape(data_temp_input.shape)
        
        plt.figure(figsize = (10,4), dpi = 200)
        plt.plot(data_temp_input, label = 'Hive Core Temperature', color = 'blue')
        plt.plot(sig_min, label = 'Forecasted Core Temp', color = 'red') 
        plt.xlabel('Samples per hour')
        plt.ylabel('Temperature in celcius')
        plt.legend()
        plt.show()
        param_min_h,param_min_theta = [], []
        for m in range(7):
            param_min_h.append(result.params['h_'+str(m)].value)
            param_min_theta.append(result.params['theta_'+str(m)].value)
            
        ##one day ahead forecast 
        forecast_h = param_min_h[-1]
        forecast_theta = param_min_theta[-1]
        h = forecast_h
        theta_ideal = forecast_theta + 273.15
        #theta_ideal = result.params['theta_'+str(6)].value + 273.15
        t = np.linspace(0,23, num = 24) #forecast hourly
        a = [1]*len(t)
        r_c = 115#29#367 #ps['w1'].value
        r_h = 1094#280 #1151 #ps['w2'].value
        
        env_temp_future = data_all[argv[1]][7*24:21*24].values + 273.15
        solar_rad_future = data_all[argv[2]][7*24:21*24].values
        hive_temp_future = data_all[argv[3]][7*24:21*24].values
        
        forecasted = []
        
        for n in range(14):
            forecasted = np.concatenate((forecasted, run_bee_eq(t, a, r_c, r_h, env_temp_future[n*24:(n+1)*24], h, 
                                         solar_rad_future[n*24:(n+1)*24], 
                                         k_W, k_A_W, k_hive, eta_W, eta_A_W, eta_B_W, theta_ideal) - 273.15))
        
        for n in forecasted:
            print(n)
        plt.figure(figsize = (10,4), dpi = 200)
        plt.plot(hive_temp_future, label = 'Hive Core Temperature', color = 'blue')
        plt.plot(forecasted, label = 'Forecasted Core Temp', color = 'red') 
        plt.xlabel('Samples per hour')
        plt.ylabel('Temperature in celcius')
        plt.legend()
        plt.show()
        forecasted_all = np.concatenate((forecasted_all, forecasted))
    
    return forecasted_all

In [ ]:
if __name__ == "__main__":
    filename = 'C:\Bee Temp Data\C284_w_Env.csv'
    environment_temperature = 'Air Temp'
    solar_radiation = 'Sol Rad'
    hive_temperature = 'Temp'
    all_temp = main(filename, environment_temperature, solar_radiation, hive_temperature)